# Using an IBM Quantum Computer Over the Cloud
***

Here we show you how to inferface with actual quantum hardware over the cloud!


### Setup

1. Go [here](https://quantum-computing.ibm.com/) and create an account. 
2. Once in your account, go to your dashboard, and copy your API access token. This allows you to access machines with up to 15 qubits.
3. Copy and paste that token below, then execute the code. You might have to wait in a queue before the computation starts. 
4. [Install Qiskit](https://qiskit.org/documentation/install.html)

See [here](https://github.com/XanaduAI/pennylane-qiskit) for more information on how to use Xanadu's PennyLane with the IBM Q Experience. 


In [ ]:
# IBMQX_API_TOKEN = "paste_your_token_here"

### Can we run the Quantum Neural Network from the other notebook?

Unfortunately that code is incompatible with the IBM quantum hardware, because it uses a different architecture. The Xanadu photonic quantum computer (which is not yet available) uses continous-variable states (qubits can take on any real-valued number) and operates as an analog quantum computer (does not use the quantum digital gate model).

The IBM quantum hardware is built using binary (two-state) qubits and operates according to the gate model. 

As of January 2020, there is only one example of implementing a quantum neural network on a gate-based quantum computer, and the implementation isn't straightforward. It's not that the gates are difficult to implement. Rather, the way they formulated the problem is complex, and computing the gradients and weight updates is non-trivial. The experiment can be found here: https://www.nature.com/articles/s41534-019-0140-4.pdf

This involved implementing the set of quantum gates shown below:

<img src="./images/quantum-perceptron-gates.png" alt="QNN" style="width: 500px;"/>


While it's too complex to reproduce this work for the purposes of this tutorial, we can perform a simpler calculation using a subset of the gates above. At least this will give us some experience in operating the quantum hardware. 

**However** if anyone wants to try to reproduce this as a passion project, slack me and we can work on it together. 


***
## Performing Simple Gate Manipulations with Qiskit

Here we show how to entangle two qubits, measure their values, and read out the results. Our goal is to create the state:

$\vert \Psi \rangle = \frac{1}{\sqrt{2}}[\vert 00 \rangle + \vert11 \rangle]$ 

Where both qubits are in a superposition of `0` and `1`, but can't have opposite values.


In [ ]:
# imports 
from qiskit import *
import matplotlib.pyplot as plt
%matplotlib inline 

### Building the Circuit

We define a quantum circuit with two qubits and two outputs. The outputs are treated as a register of classical bits which will hold the scalar value of any measurements we perform on the qubits. 

In [ ]:
circuit = QuantumCircuit(2,2, name='test')  # define a quantum circuit with two inputs and two outputs
circuit.draw(output='mpl')

The algorithm for entangling two qubits is a 'Hadamard Gate' on one, followed by a controlled-not (CNOT) gate between both. Then we have to measure the result. 

In [ ]:
circuit.h(0) # hadamard gate on qubit 0
circuit.cx(0,1) # CNOT gate between qubit 0 and qubit 1
circuit.measure([0,1], [0,1]) # qubits [0,1] are measured and results are stored in classical bits [0,1] in order
circuit.draw(output='mpl')

### Running on a Simulator

In [ ]:
from qiskit.visualization import plot_histogram

simulator = Aer.get_backend('qasm_simulator')
result = execute(circuit, backend=simulator).result()
plot_histogram(result.get_counts(circuit))

The simulator also includes realistic device noise, which is why we obtain imperfect results. 

### Running on Quantum Hardware

For more details on interfacing with quantum hardware, see [here](https://medium.com/qiskit/qiskit-backends-what-they-are-and-how-to-work-with-them-fb66b3bd0463).

You'll need your IMB quantum experience API token to set up the interface with the hardware:


In [ ]:
IBMQ.enable_account(IBMQX_API_TOKEN)
provider = IBMQ.get_provider(hub = 'ibm-q')

Let's grab the least busy backend that has at least the minimum number of qubits we need:

In [ ]:
num_qubits = 2

from qiskit.providers.ibmq import least_busy
possible_devices = provider.backends(filters=lambda x: 
                                     x.configuration().n_qubits >= num_qubits
                                       and 
                                     x.configuration().simulator == False)
qcomp = least_busy(possible_devices)
print(qcomp)

Now let's run our job. 

In [ ]:
import qiskit.tools.jupyter
from qiskit.tools.monitor import job_monitor
%qiskit_job_watcher

job = execute(circuit, backend=qcomp)
job_monitor(job)

Viewing the results:

In [ ]:
%qiskit_disable_job_watcher

result = job.result()
plot_histogram(result.get_counts(circuit))


You can see our results were not perfect, and this is in part due to noise in the device which corrupts the quantum states. 

The devices we have today are 'kind of quantum', but not perfect. There are techniques being devloped to correct errors. But in the meantime, there are a lot of interest in finding tasks that can still obtain a quantum speedup even with noisy, small-scale quantum processors. 

***
***

## Challenge: Implementing the "Rotoselect" Algorithm

Try executing the code from this tutorial on real hardware and let us know if you succeed!
https://pennylane.ai/qml/app/tutorial_rotoselect.html

To access the IBM device from PennyLane, you'll need to define your device as follows, where `QUANTUM_BACKEND` gives the name of the hardware device you want to use (e.g. `ibmq_burlington`). 

```
device = qml.device('qiskit.ibmq', wires=1, backend=QUANTUM_BACKEND, ibmqx_token=IBMQX_API_TOKEN)
```
